In [8]:
import os
import cv2
import json
import numpy as np
import pandas as pd
import tifffile as tiff
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt

In [9]:
df = pd.read_csv("../data/tile_meta.csv")

In [10]:
df = df[df["dataset"] == 3]
tiles_dicts = df["id"]

In [11]:
def make_seg_mask(tiles_dict):
    mask = np.zeros((512, 512), dtype=np.float32)
    for annot in tiles_dict['annotations']:
        cords = annot['coordinates']
        if annot['type'] == "blood_vessel":
            for cd in cords:
                rr, cc = np.array([i[1] for i in cd]), np.asarray([i[0] for i in cd])
                mask[rr, cc] = 1
                
    contours,_ = cv2.findContours((mask*255).astype(np.uint8), 1, 2)
    zero_img = np.zeros([mask.shape[0], mask.shape[1]], dtype="uint8")

    for p in contours:
        cv2.fillPoly(zero_img, [p], (255, 255))
    # zero_img = cv2.blur(zero_img, (6, 6))
    return zero_img    

In [12]:
def make_seg_mask_3class(tiles_dict):
    mask = np.zeros((512, 512), dtype=np.float32)
    contour_dict = {'blood_vessel': [], 'glomerulus': [], 'unsure': []} # 追加
    for annot in tiles_dict['annotations']:
        cords = annot['coordinates']
        if annot['type'] in contour_dict: # 追加
            for cd in cords:
                rr, cc = np.array([i[1] for i in cd]), np.asarray([i[0] for i in cd])
                mask[rr, cc] = 1
                contour_dict[annot['type']].append(mask.copy()) # 各クラスのcontourを記録
                mask[rr, cc] = 0 # maskのリセット

    colors = [(1, 0, 0), (0, 1, 0), (0, 0, 1)] # 各クラスに対する色を定義 (ここではRGBで赤、緑、青)
    zero_img = np.zeros([mask.shape[0], mask.shape[1], 3], dtype="uint8")

    for idx, class_type in enumerate(contour_dict.keys()): # 各クラスのcontourについて
        for class_mask in contour_dict[class_type]: # 各クラスのmaskについて
            contours, _ = cv2.findContours((class_mask).astype(np.uint8), 1, 2) # 各クラスのmaskからcontourを抽出

            for p in contours: # 抽出したcontourに対して色を割り当て
                cv2.fillPoly(zero_img, [p], colors[idx]) # ここで指定した色で描画

    return zero_img

In [13]:
all_ids = []

In [19]:
os.makedirs('../data/train_images_dataset3', exist_ok=True)
for i, tldc in enumerate(tqdm(tiles_dicts)):
    array = tiff.imread(f'../data/train/{tldc}.tif')
    img_example = Image.fromarray(array)
    img = np.array(img_example)
    # mask = make_seg_mask(tldc)
    # plt.imshow(mask)
    # break
    cv2.imwrite(f'../data/train_images_dataset3/{tldc}.png', img)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5400/5400 [01:09<00:00, 77.25it/s]
